In [1]:
#import libraries
import pandas as pd
import sqlite3

In [2]:

# read csv files
df_day1 = pd.read_csv(r"C:\Users\Kanika\Desktop\data-transformation-assignment\data-transformation-assignment\SQL BASED\products_day1.csv")
df_day2 = pd.read_csv(r"C:\Users\Kanika\Desktop\data-transformation-assignment\data-transformation-assignment\SQL BASED\products_day2.csv")



In [3]:
#connect to inmemory databse 
conn = sqlite3.connect(":memory:")


In [4]:
df_day1.to_sql("products_day1", conn, index=False, if_exists="replace")
df_day2.to_sql("products_day2", conn, index=False, if_exists="replace")

6

In [5]:
#- Identify full rows that were ADDED or REMOVED.

query1 = """
SELECT d2.product_id, d2.name, d2.category, d2.price, d2.stock, 'ADDED' AS change_type
FROM products_day2 d2
WHERE d2.product_id NOT IN (SELECT product_id FROM products_day1)

UNION ALL

SELECT d1.product_id, d1.name, d1.category, d1.price, d1.stock, 'REMOVED' AS change_type
FROM products_day1 d1
WHERE d1.product_id NOT IN (SELECT product_id FROM products_day2)
"""

result_query1 = pd.read_sql_query(query1, conn)
print(result_query1)


   product_id         name    category  price  stock change_type
0         107  Smart Watch   Wearables  99.00     50       ADDED
1         106     Notebook  Stationery   2.99    300     REMOVED


In [6]:
#- Compare matching rows and detect column-level changes (e.g., price, stock).

query2 = """
SELECT d1.product_id, 'price' AS column_changed, d1.price AS old_value, d2.price AS new_value
FROM products_day1 d1
JOIN products_day2 d2 ON d1.product_id = d2.product_id
WHERE d1.price != d2.price

UNION ALL

SELECT d1.product_id, 'stock' AS column_changed, d1.stock AS old_value, d2.stock AS new_value
FROM products_day1 d1
JOIN products_day2 d2 ON d1.product_id = d2.product_id
WHERE d1.stock != d2.stock
"""

result_query2 = pd.read_sql_query(query2, conn)
print(result_query2)


   product_id column_changed  old_value  new_value
0         101          price      25.99      23.99
1         105          price      35.00      37.00
2         102          stock     200.00     180.00
